# Video Semantic Search with SigLIP 2

Search inside videos using natural language queries powered by **SigLIP 2** + **ChromaDB**

## Architecture

```
INDEXING PIPELINE (Offline)
┌─────────────────────────────────────────────────────────────────────────────┐
│  Video File ──► OpenCV ──► Frames + Timestamps ──► SigLIP2 ──► ChromaDB    │
│                 extract      (sampled @ 1fps)       embed      store        │
└─────────────────────────────────────────────────────────────────────────────┘

QUERY PIPELINE (Online)
┌─────────────────────────────────────────────────────────────────────────────┐
│  Text Query ──► SigLIP2 ──► Query Vector ──► ChromaDB ──► Top-K Results    │
│                 embed                         search      + timestamps      │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Tech Stack

| Component | Technology | Purpose |
|-----------|------------|---------|
| Frame Extraction | OpenCV | Read video, extract frames at configurable FPS |
| Embedding Model | SigLIP 2 (google/siglip2-base-patch16-256) | Image & text embeddings in shared 768-dim space |
| Vector Database | ChromaDB | Store & search embeddings with HNSW index |
| Deep Learning | PyTorch + Transformers | Model inference with GPU acceleration |

## 1. Setup & Installation

In [2]:
# Install dependencies (uncomment if needed)
# %pip install torch transformers Pillow opencv-python chromadb numpy tqdm matplotlib
!uv add torch transformers Pillow opencv-python chromadb numpy tqdm matplotlib

Using CPython 3.12.4 interpreter at: C:\Users\kirub\AppData\Local\Programs\Python\Python312\python.exe
Creating virtual environment at: .venv
Resolved 117 packages in 4.50s
Prepared 60 packages in 1m 52s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 97 packages in 15.05s
 + annotated-types==0.7.0
 + anyio==4.12.1
 + attrs==25.4.0
 + backoff==2.2.1
 + bcrypt==5.0.0
 + build==1.4.0
 + certifi==2026.1.4
 + charset-normalizer==3.4.4
 + chromadb==1.4.1
 + click==8.3.1
 + colorama==0.4.6
 + coloredlogs==15.0.1
 + contourpy==1.3.3
 + cycler==0.12.1
 + distro==1.9.0
 + durationpy==0.10
 + filelock==3.20.3
 + flatbuffers==25.12.19
 + fonttools==4.61.1
 + fsspec==2026.1.0
 + googleapis-common-protos==1.72.0
 + grpcio==1.76.0
 + h11==0.16.0
 + hf-xet==1.2.0
 + httpcore==1.0.9
 + httptools==0.7.1
 + httpx==0.28.

In [ ]:
import cv2
import torch
import numpy as np
from pathlib import Path
from datetime import timedelta
from PIL import Image
from tqdm import tqdm
import chromadb
from transformers import AutoProcessor, AutoModel
import matplotlib.pyplot as plt

# Check device - prefer CUDA for faster inference
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

Using device: cpu


## 2. Configuration

All configurable parameters in one place with justifications.

In [ ]:
# =============================================================================
# MODEL CONFIGURATION
# =============================================================================

# SigLIP 2 base model - fastest variant with 768-dim embeddings
# Alternatives: siglip2-large-patch16-384 (1024-dim), siglip2-so400m-patch14-384 (1152-dim)
MODEL_NAME = "google/siglip2-base-patch16-256"

# Embedding dimension for base model (changes with model variant)
EMBEDDING_DIM = 768

# =============================================================================
# FRAME EXTRACTION CONFIGURATION  
# =============================================================================

# Sample rate in frames per second
# 1.0 = extract 1 frame every second (good balance of coverage vs storage)
# 2.0 = extract 2 frames per second (more granular, 2x storage)
# 0.5 = extract 1 frame every 2 seconds (less granular, half storage)
SAMPLE_RATE = 1.0

# =============================================================================
# EMBEDDING CONFIGURATION
# =============================================================================

# Batch size for processing frames through the model
# 16 works well for 8GB VRAM with base model in float16
# Reduce to 8 if you encounter OOM errors
BATCH_SIZE = 16

# =============================================================================
# SEARCH CONFIGURATION
# =============================================================================

# Number of results to return from similarity search
TOP_K = 10

# =============================================================================
# STORAGE CONFIGURATION
# =============================================================================

# Directory structure
DATA_DIR = Path("./data")
VIDEOS_DIR = DATA_DIR / "videos"
DB_DIR = DATA_DIR / "chroma_db"

# Create directories
VIDEOS_DIR.mkdir(parents=True, exist_ok=True)
DB_DIR.mkdir(parents=True, exist_ok=True)

print(f"Model: {MODEL_NAME}")
print(f"Sample rate: {SAMPLE_RATE} fps")
print(f"Batch size: {BATCH_SIZE}")
print(f"Videos directory: {VIDEOS_DIR}")
print(f"Database directory: {DB_DIR}")

Model: google/siglip2-base-patch16-256
Sample rate: 1.0 fps
Batch size: 16
Videos directory: data/videos
Database directory: data/chroma_db


## 3. Load SigLIP 2 Model

Load the vision-language encoder with memory optimizations:
- **float16**: Reduces memory usage by 50% with minimal quality loss
- **SDPA**: Scaled Dot-Product Attention for efficient GPU computation

In [ ]:
print(f"Loading model: {MODEL_NAME}")

# Load processor (handles image preprocessing and text tokenization)
processor = AutoProcessor.from_pretrained(MODEL_NAME)

# Load model with optimizations
# - torch_dtype=float16: Half precision for 50% memory savings
# - attn_implementation="sdpa": PyTorch's efficient attention
# - device_map="auto": Automatically place on GPU if available
model = AutoModel.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    attn_implementation="sdpa",
    device_map="auto"
).eval()  # Set to evaluation mode (disables dropout)

print(f"Model loaded successfully!")
print(f"Model device: {model.device}")
print(f"Image embedding dimension: {model.config.vision_config.hidden_size}")

Loading model: google/siglip2-base-patch16-256


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

Model loaded successfully!
Model device: cpu
Image embedding dimension: 768


## 4. Frame Extraction

Extract frames from video at a configurable sample rate using OpenCV.

**Key formula**: `timestamp_seconds = frame_index / fps`

In [ ]:
def extract_frames(video_path: str, sample_rate: float = SAMPLE_RATE):
    """
    Extract frames from video at specified sample rate.
    
    Args:
        video_path: Path to video file
        sample_rate: Frames per second to extract (1.0 = 1 frame/sec)
    
    Returns:
        frames: List of RGB frames (numpy arrays)
        metadata: List of dicts with timestamp info
        video_info: Dict with video properties
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = total_frames / fps if fps > 0 else 0
    
    video_info = {
        "fps": fps,
        "total_frames": total_frames,
        "width": width,
        "height": height,
        "duration": duration
    }
    
    print(f"Video Info:")
    print(f"  - FPS: {fps:.2f}")
    print(f"  - Total Frames: {total_frames}")
    print(f"  - Resolution: {width}x{height}")
    print(f"  - Duration: {timedelta(seconds=int(duration))}")
    
    # Calculate sampling interval
    # If video is 30fps and sample_rate is 1.0, we take every 30th frame
    frame_interval = max(1, int(fps / sample_rate))
    expected_frames = total_frames // frame_interval
    print(f"  - Sampling: 1 frame every {frame_interval} frames ({sample_rate} fps)")
    print(f"  - Expected extracted frames: ~{expected_frames}")
    
    frames = []
    metadata = []
    frame_idx = 0
    
    pbar = tqdm(total=total_frames, desc="Extracting frames")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_idx % frame_interval == 0:
            # Convert BGR (OpenCV default) to RGB (what models expect)
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
            
            # Calculate timestamp: frame_index / fps = seconds
            timestamp = frame_idx / fps
            
            metadata.append({
                "frame_idx": frame_idx,
                "timestamp": timestamp,
                "timestamp_str": str(timedelta(seconds=int(timestamp))),
            })
        
        frame_idx += 1
        pbar.update(1)
    
    pbar.close()
    cap.release()
    
    print(f"\nExtracted {len(frames)} frames")
    return frames, metadata, video_info


def get_frame_at_index(video_path: str, frame_idx: int):
    """Retrieve a single frame from video at specific index (for displaying results)."""
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return None

## 5. Embedding Functions

Generate embeddings using SigLIP 2:
- **Images**: Batch process frames through vision encoder
- **Text**: Encode queries through text encoder

Both produce vectors in the same 768-dimensional space, enabling cross-modal similarity search.

In [ ]:
def embed_images(frames: list, batch_size: int = BATCH_SIZE):
    """
    Generate embeddings for a list of frames using SigLIP 2.
    
    Args:
        frames: List of RGB numpy arrays
        batch_size: Number of frames to process at once (reduce if OOM)
    
    Returns:
        embeddings: numpy array of shape (n_frames, EMBEDDING_DIM)
    """
    all_embeddings = []
    
    for i in tqdm(range(0, len(frames), batch_size), desc="Generating embeddings"):
        batch = frames[i:i + batch_size]
        
        # Process batch through the processor (resizes to 256x256, normalizes)
        inputs = processor(images=batch, return_tensors="pt").to(model.device)
        
        with torch.no_grad():
            # Get image features from vision encoder
            embeddings = model.get_image_features(**inputs)
            
            # L2 normalize embeddings (required for cosine similarity)
            # This projects all vectors onto the unit sphere
            embeddings = embeddings / embeddings.norm(dim=-1, keepdim=True)
        
        all_embeddings.append(embeddings.cpu())
    
    return torch.cat(all_embeddings, dim=0).numpy()


def embed_text(query: str):
    """
    Generate embedding for a text query using SigLIP 2.
    
    Args:
        query: Natural language search query
    
    Returns:
        embedding: numpy array of shape (1, EMBEDDING_DIM)
    """
    # IMPORTANT: Use padding="max_length" because SigLIP was trained this way
    # Without this, results may be suboptimal
    inputs = processor(
        text=[query], 
        padding="max_length",  # Critical: model was trained with max_length padding
        return_tensors="pt"
    ).to(model.device)
    
    with torch.no_grad():
        # Get text features from text encoder
        embedding = model.get_text_features(**inputs)
        
        # L2 normalize (same as images)
        embedding = embedding / embedding.norm(dim=-1, keepdim=True)
    
    return embedding.cpu().numpy()

## 6. ChromaDB Vector Database

Set up persistent vector storage with HNSW index for fast similarity search.

**Why ChromaDB?**
- Simple Python API
- Automatic HNSW indexing (O(log n) search)
- Persistent storage to disk
- Metadata filtering support

In [ ]:
# Initialize ChromaDB with persistent storage
# Data is automatically saved to disk at DB_DIR
chroma_client = chromadb.PersistentClient(path=str(DB_DIR))

# Create or get collection for video frames
# Using cosine similarity (appropriate for normalized embeddings)
collection = chroma_client.get_or_create_collection(
    name="video_frames",
    metadata={"hnsw:space": "cosine"}  # Cosine distance for similarity search
)

print(f"Collection: {collection.name}")
print(f"Current frame count: {collection.count()}")


def get_database_stats():
    """Get statistics about the indexed database."""
    total_frames = collection.count()
    
    print("\nDatabase Statistics")
    print("=" * 40)
    print(f"Total indexed frames: {total_frames}")
    
    if total_frames > 0:
        # Get unique video IDs
        all_data = collection.get(include=["metadatas"])
        video_ids = set(m.get("video_id", "unknown") for m in all_data["metadatas"])
        print(f"Total videos: {len(video_ids)}")
        print(f"\nIndexed videos:")
        for v in video_ids:
            print(f"  - {v}")
    print("=" * 40)


def clear_database():
    """Clear all data from the collection (use with caution!)."""
    global collection
    chroma_client.delete_collection("video_frames")
    collection = chroma_client.create_collection(
        name="video_frames",
        metadata={"hnsw:space": "cosine"}
    )
    print("Database cleared!")

Collection: video_frames
Current frame count: 0


## 7. Video Indexing Pipeline

The main indexing function that orchestrates:
1. Frame extraction (OpenCV)
2. Embedding generation (SigLIP 2)
3. Vector storage (ChromaDB)

In [ ]:
def index_video(video_path: str, video_id: str, sample_rate: float = SAMPLE_RATE):
    """
    Index a video into ChromaDB for semantic search.
    
    Args:
        video_path: Path to video file
        video_id: Unique identifier for this video (used in search results)
        sample_rate: Frames per second to extract
    
    Returns:
        num_frames: Number of frames indexed
    """
    print(f"\n{'='*60}")
    print(f"Indexing video: {video_id}")
    print(f"{'='*60}\n")
    
    # Step 1: Extract frames from video
    frames, metadata, video_info = extract_frames(video_path, sample_rate)
    
    if len(frames) == 0:
        raise ValueError("No frames extracted from video")
    
    # Step 2: Generate embeddings for all frames
    print("\nGenerating embeddings...")
    embeddings = embed_images(frames)
    print(f"Embeddings shape: {embeddings.shape}")
    
    # Step 3: Prepare data for ChromaDB
    # Create unique IDs for each frame
    ids = [f"{video_id}_frame_{m['frame_idx']}" for m in metadata]
    
    # Add video info to metadata
    for m in metadata:
        m["video_id"] = video_id
        m["video_path"] = str(video_path)
    
    # Step 4: Store in ChromaDB
    print("\nStoring in ChromaDB...")
    collection.add(
        ids=ids,
        embeddings=embeddings.tolist(),
        metadatas=metadata
    )
    
    print(f"\nSuccessfully indexed {len(frames)} frames from '{video_id}'")
    print(f"Total frames in database: {collection.count()}")
    
    return len(frames)

## 8. Search Functions

Search indexed videos using natural language queries.

In [ ]:
def search(query: str, top_k: int = TOP_K):
    """
    Search for frames matching the text query.
    
    Args:
        query: Natural language search query (e.g., "a person writing on whiteboard")
        top_k: Number of results to return
    
    Returns:
        results: List of dicts with video_id, timestamp, score, etc.
    """
    print(f"\nSearching for: '{query}'")
    
    # Embed the query text
    query_embedding = embed_text(query)
    
    # Search ChromaDB
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=top_k,
        include=["metadatas", "distances"]
    )
    
    # Format results
    search_results = []
    for i in range(len(results["ids"][0])):
        meta = results["metadatas"][0][i]
        distance = results["distances"][0][i]
        
        # Convert cosine distance to similarity score
        # ChromaDB returns distance (0 = identical, 2 = opposite)
        # We convert to similarity (1 = identical, -1 = opposite)
        score = 1 - distance
        
        search_results.append({
            "rank": i + 1,
            "video_id": meta["video_id"],
            "video_path": meta.get("video_path", ""),
            "timestamp": meta["timestamp"],
            "timestamp_str": meta["timestamp_str"],
            "frame_idx": meta["frame_idx"],
            "score": score,
        })
    
    print(f"Found {len(search_results)} results\n")
    return search_results


def display_search_results(results, max_display: int = 5):
    """
    Display search results with frame thumbnails.
    
    Args:
        results: List of result dicts from search()
        max_display: Maximum number of results to display
    """
    if not results:
        print("No results to display")
        return
    
    # Print text results
    print("Results:")
    print("-" * 70)
    for r in results[:max_display]:
        print(f"#{r['rank']} | {r['video_id']} | Time: {r['timestamp_str']} | Score: {r['score']:.4f}")
    print("-" * 70)
    
    # Load and display frames
    frames = []
    titles = []
    
    for r in results[:max_display]:
        frame = get_frame_at_index(r["video_path"], r["frame_idx"])
        if frame is not None:
            frames.append(frame)
            titles.append(f"#{r['rank']} {r['timestamp_str']}\nScore: {r['score']:.3f}")
    
    if frames:
        # Create figure with subplots
        n = len(frames)
        cols = min(n, 5)
        rows = (n + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
        if n == 1:
            axes = [axes]
        else:
            axes = axes.flatten() if rows > 1 else axes
        
        for i, ax in enumerate(axes):
            if i < n:
                ax.imshow(frames[i])
                ax.set_title(titles[i], fontsize=9)
            ax.axis('off')
        
        plt.tight_layout()
        plt.show()

---

# Demo Usage

Now let's use the pipeline to index and search videos.

## Index a Video

Update the `VIDEO_PATH` below to point to your video file.

In [ ]:
# =============================================================================
# UPDATE THESE VALUES FOR YOUR VIDEO
# =============================================================================

VIDEO_PATH = "./data/videos/video.mp4"  # Path to your video file
VIDEO_ID = "sample_video"                 # Unique identifier for this video

# =============================================================================

# Check if video exists and index it
if Path(VIDEO_PATH).exists():
    num_frames = index_video(VIDEO_PATH, VIDEO_ID, sample_rate=SAMPLE_RATE)
else:
    print(f"Video not found: {VIDEO_PATH}")
    print("\nPlease update VIDEO_PATH to point to your video file.")
    print(f"You can place videos in: {VIDEOS_DIR.absolute()}")

Video not found: ./data/videos/video.mp4

Please update VIDEO_PATH to point to your video file.
You can place videos in: /content/data/videos


## Check Database Stats

In [ ]:
get_database_stats()

## Search Videos

Try different natural language queries to find matching moments in your indexed videos.

In [ ]:
# Example search query - update this to match your video content
query = "a person talking"

results = search(query, top_k=5)
display_search_results(results)

In [ ]:
# Try another query
query = "outdoor scene with nature"

results = search(query, top_k=5)
display_search_results(results)

## Batch Search (Multiple Queries)

In [ ]:
# Test multiple queries at once
test_queries = [
    "a person smiling",
    "text on screen",
    "close up of face",
    "wide shot",
    "bright colors",
]

for q in test_queries:
    results = search(q, top_k=3)
    if results:
        print(f"Query: '{q}'")
        for r in results:
            print(f"  -> {r['timestamp_str']} (score: {r['score']:.3f})")
        print()

---

## Clean Up (Optional)

In [ ]:
# Uncomment to clear all indexed data
# clear_database()

---

## Summary

This notebook implements a complete video semantic search pipeline:

| Step | Function | Description |
|------|----------|-------------|
| 1 | `extract_frames()` | Extract frames from video at configurable FPS using OpenCV |
| 2 | `embed_images()` | Generate 768-dim SigLIP 2 embeddings for frames |
| 3 | `embed_text()` | Generate 768-dim SigLIP 2 embedding for text query |
| 4 | `index_video()` | Full indexing pipeline (extract -> embed -> store) |
| 5 | `search()` | Find matching frames using cosine similarity |
| 6 | `display_search_results()` | Visualize results with thumbnails |

### Key Parameters

| Parameter | Value | Purpose |
|-----------|-------|---------|
| `MODEL_NAME` | `google/siglip2-base-patch16-256` | Vision-language encoder |
| `SAMPLE_RATE` | `1.0` | Frames per second to extract |
| `BATCH_SIZE` | `16` | Frames per embedding batch |
| `TOP_K` | `10` | Search results to return |

### Next Steps

1. Index more videos to build a larger searchable collection
2. Experiment with different sample rates for your use case
3. Build a Streamlit app with video upload and playback